In [15]:
import sys
sys.path.append('./python/')

import numpy as np
from datetime import datetime
import astropy.units as u
from astropy import coordinates
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time
from sunpy import coordinates as coord
from refractivity import refractivity
import sunpy as sp
from scipy import interpolate
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import fits

print("All modules loaded")


All modules loaded


# 1) Getting atmospheric density

In [16]:
def atmospheric_density(temperature=20*u.deg_C, pressure_pa=100000*u.Pa, 
                        humidity=75, xc=380, force_xw=0, water_vapor=False, 
                        dry_air=False, verbose=0):
    
    """Return the atmospheric density

    Typical parameters for atmospheric values set as defaults. 
    """
    
    TT_C = temperature.value
    TT_K = TT_C + 273.15
    pressure_pa = pressure_pa.to(u.Pa).value #keep units in pascals
    humidity_partial = humidity/100.
    if verbose >= 2:
        print("Temp: ", TT_C, "°C \nPressure: ", pressure_pa, " Pa \nHumidity: ",\
              humidity, "% \nCO2: ", xc, " ppm")
    
    #*************** Constants ****************
    #from Ciddor 1996, Appendix A:
    AA = 1.2378847*10**-5    #K^(-2)
    BB = -1.9121316*10**-2   #K^(-2)
    CC = 33.93711047         #
    DD = -6.3431645*10**3    #K
    
    alpha = np.float64(1.00062)#
    beta  = np.float64(3.14 * 10**-8)  #Pa^(-1)
    gamma = np.float(5.6 * 10**-7)    #°C^(-2)
    
    a0 = 1.58123*10**-6      #K Pa^(-1)
    a1 = -2.9331*10**-8      #Pa^(-1)
    a2 = 1.1043*10**-10      #K^(-1) Pa^(-1)
    b0 = 5.707*10**-6        #K Pa^(-1)
    b1 = -2.051*10**-8       #Pa^(-1)
    c0 = 1.9898*10**-4       #K Pa^(-1)
    c1 = -2.376*10**-6       #Pa^(-1)
    d  = 1.83*10**-11        #K^2 Pa^(-2)
    e  = -0.765*10**-8       #K^2 Pa^(-2)
    
    #from Ciddor 1995, Section 3
    #gas constant:
    R  = 8.314510            #J mol^(-1) K^(-1)
    #molar mass of water vapor:
    Mw = 0.018015            #kg/mol
    #molar mass of dry air containing a CO2 concentration of xc ppm:
    Malpha = (10**-3)*(28.9635 + (12.011*10**-6)*(xc-400)) 
    
    #***************End Constants*****************
    
    #saturation vapor pressure of water vapor in air at temperature T, from Ciddor 1996 Section 3:
    svp = np.exp(AA*TT_K**2 + BB*TT_K + CC + DD/TT_K)
    
    #enhancement factor of water vapor in air, whatever that is:
    f = alpha + beta*pressure_pa + gamma*TT_C**2
    
    if force_xw == 0:
        xw = f*humidity_partial*svp/pressure_pa
    else:
        xw=force_xw #molar fraction of water vapor
        
    #from Ciddor 1996 Appendix:    
    Z=1-(pressure_pa/TT_K)*(a0 + a1*TT_C + a2*TT_C**2 + (b0 + b1*TT_C)*xw 
                            + (c0 + c1*TT_C)*xw**2) + ((pressure_pa/TT_K)**2)*(d + e*xw**2)
    
    if water_vapor:
        density = (pressure_pa*Mw*xw/(Z*R*TT_K))
    elif dry_air:
        density = (pressure_pa * Malpha/(Z*R*TT_K))*(1 - xw)
    else:
        density = (pressure_pa * Malpha/(Z*R*TT_K))*(1 - xw*(1 - Mw/Malpha))
    
    if verbose >= 2:
        print("svp: ", svp, '\nf: ', f, '\nxw: ', xw, '\nZ: ', Z, '\ndensity: ', density)
    
    atmosphere_values = {'R':R, 'Z':Z, 'Ma':Malpha, 'Mw':Mw, 'svp':svp, 'f':f, 'density':density,\
                         'TT_C':TT_C, 'TT_K':TT_K, 'pressure_pa':pressure_pa, 'humidity':humidity, 'xw':xw}

    return(density)


In [17]:
atmospheric_density()

1.1808468323708414

# 2) Use density to get refractivity

In [18]:
def refractivity(wavelength_nm=np.array([633.0])*u.nm, temperature=20*u.deg_C, 
                 pressure_pa=100000*u.Pa, humidity=75, xc=380, verbose=0):
    
    """Return the refractivity at a given wavelength

    Typical parameters for atmospheric values set as defaults. Note that refractivity  = index of refraction - 1
    """

    
    wavelength_mic = wavelength_nm.to(u.micron).value
    
    #convert wavelengths in air to vacuum wavelengths [lambda(air) = lambda(vacuum)/n(air)]
    #using mean index of refraction of air = 1.00027
    wavelength_vac = wavelength_mic*1.00027
    wavenumber = 1/wavelength_vac
    
    #*****************  Constants  ******************
    #from Ciddor 1996, Appendix A
    #originally from Peck and Reeder 1962:
    k0 = 238.0185   #microns^(-2)
    k1 = 5792105.   #microns^(-2)
    k2 = 57.362     #microns^(-2)
    k3 = 167917.    #microns^(-2)
    
    #originally from Owens 1967:
    w0 = 295.235    #microns^(-2)
    w1 = 2.6422     #microns^(-2)
    w2 = -0.032380  #microns^(-4)
    w3 = 0.004028   #microns^(-6)
    #*************  End Constants  ******************
    
    #refractivity of air at 15°C, 101325 Pa, 0% humidity, and a fixed 450 ppm of CO2
    #from Ciddor 1996 Eq. 1:
    nas = (10**-8)*(k1/(k0 - wavenumber**2) + k3/(k2 - wavenumber**2)) + 1
    
    #refractivity of air at 15°C, 101325 Pa, 0% humidity, and a variable xc pmm of CO@
    #from Ciddor 1996 Eq. 2:
    naxs = (nas - 1)*(1 + (0.534*10**-6)*(xc - 450)) + 1
    
    #refractivity of water vapor at 20°C, 1333 Pa, 0% humidity
    #correction actor derived by Ciddor 1996 by fitting to measurements:
    cf = 1.022
    #from Ciddor 1996 Eq. 3:
    nws = (10**-8)*cf*(w0 + w1*wavenumber**2 + w2*wavenumber**4 + w3*wavenumber**6) + 1
    
    #density of dry air at standard conditions:
    density_axs = atmospheric_density(15*u.deg_C, 101325*u.Pa, 0, xc, dry_air=True)
    #density of water vapor at standard conditions:
    density_ws  = atmospheric_density(20*u.deg_C, 1333*u.Pa, 100, xc, force_xw=1)
    
    #density of dry air at input conditions:
    density_a = atmospheric_density(temperature, pressure_pa, humidity, xc, dry_air=True)
    #density of water vapor at input conditions:
    density_w = atmospheric_density(temperature, pressure_pa, humidity, xc, water_vapor=True)
    if verbose >= 1:
        print("density a - ",density_a,density_axs,density_a/density_axs)
        print("density w - ",density_w,density_ws,density_w/density_ws)
    
    #from Ciddor 1996 Eq. 5:
    nprop_a = (density_a/density_axs)*(naxs - 1)
    nprop_w = (density_w/density_ws)*(nws - 1)
    nprop = nprop_a + nprop_w
    
    if verbose >= 1:
        print("n(axs): ", (naxs - 1)*10**8, "\nn(ws): ", (nws - 1)*10**8, "\nrho(a/axs): ", 
              (density_a/density_axs), "\nrho(w/ws): ", 
              (density_w/density_ws), "\nn(prop): ", nprop*10**8)
    if verbose >= 2:
        print("n(air): ", (density_a/density_axs)*(naxs - 1)*10**8, "\nn(water): ", 
              (density_w/density_ws)*(nws - 1)*10**8)
    
    return(nprop)


In [19]:
refractivity()

array([0.00026759])

# 3) Main atmospheric refractivity function. 

In [20]:
def atmospheric_refraction(wavelength=np.array([400, 500, 600, 700, 800])*u.nm, 
                           input_times=np.array([1]), latitude=20.71*u.deg, 
                           longitude=-156.25*u.deg, altitude=3055.*u.m, 
                           air_temp=20*u.deg_C, air_pressure=100000.*u.Pa, 
                           humidity=75., co2_conc=415., verbose=0):
    """Return the wavelengths-dependent atmospheric refraction

    Typical parameters for atmospheric values set as defaults. 
    Parallactic angle in degrees and refraction magnitude in arcseconds
    are also returned. 
    
    """
    
    arcsec_conversion = np.degrees(3600.) # *** What's going on here?
    num_waves=len(wavelength)
    wavelength.astype(float) 
    
    #setting default time to now in Julian dates
    if np.alltrue(input_times == np.array([1])):  # *** input times is set to np.array([1]) by default?
        input_times = Time(np.array([datetime.utcnow()]), scale='utc')
        input_times.format = 'jd'
    else:
        input_times = Time(input_times, format='jd', scale='utc')    
    
    input_times.delta_ut1_utc = 0.0 # set offset = 0 
    
    num_times = len(input_times)
    
    refrac = refractivity(wavelength, air_temp, air_pressure, 
                          humidity, co2_conc, verbose=verbose)

    #get the Sun's RA and Dec, then print them
    sunpos = coordinates.get_sun(input_times)
    if verbose == 1:
        print(sunpos.ra,sunpos.dec,sunpos.obstime)
    
    #Local information for Haleakala
    haleakala = EarthLocation(lat=latitude, lon=longitude, height=altitude)
    local_sidereal = input_times.sidereal_time('apparent',haleakala.lon)
    
    #Get hour angle, altitude and azimuth
    #The rest of the program only uses the hour angle and altitude
    ha_all = ((local_sidereal - sunpos.ra).deg + 360)%360 * u.deg
    ha_all = ha_all.value
    frame_obstime = AltAz(obstime=input_times, location=haleakala)  # *** How is alt az different from ra and Dec?
    sunpos_altaz  = sunpos.transform_to(frame_obstime)
    alt_all = sunpos_altaz.alt.deg
    idx = (np.abs(alt_all - 0)).argmin()
    alt_all[idx] = alt_all[idx - 1]

    idx = (np.abs(alt_all - 0)).argmin()
    alt_all[idx] = alt_all[idx - 1]
    
    #continue with calculations
    beta = 0.001254*(273.15 + air_temp.value)/273.15
    coeff_a = refrac*(1 - beta)
    coeff_b = refrac*(beta - refrac/2.)
    
    #calculate the magnitude of the refraction for each time and wavelength
    refraction_calc = np.ones((num_times, num_waves))
    for wv in range (num_waves):
        refraction_wv = (coeff_a[wv]*np.tan(np.radians(90 - alt_all))) 
        - (coeff_b[wv]*(np.tan(np.radians(90 - alt_all)))**3)
        
        refraction_wv = refraction_wv*arcsec_conversion
        refraction_calc[:, wv] = refraction_wv
    #find the parallactic angle
    
    #get everything in degrees        
    parallactic_angle_sin = np.sin(np.deg2rad(ha_all))/np.sin(np.deg2rad(90 - alt_all)) *\
                            np.sin(np.deg2rad(90 - latitude.to(u.deg).value))
    parallactic_angle = np.rad2deg(np.arcsin(parallactic_angle_sin))
        
    if verbose == 1:
        print("\nInput Time(s) in Julian dates: ", input_times)
        print("\nSun's RA: ", sunpos.ra.degree)
        print("Sun's Dec: ", sunpos.dec.degree)
        print("Local Sidereal Time: ", local_sidereal)
        print('\nHour Angle: ', ha_all)
        print("Altitude: ", alt_all)
        print("Azimuth: ", sunpos_altaz.az.deg)
        print()
        for time, refractions in zip(input_times, refraction_calc):
            print("Refraction for Julian Date ", time, ": ", refractions)
        print()
        for time, angles in zip(input_times, parallactic_angle):
            print ("Parallactic Angle for Julian Date ", time, ": ", angles)
    
    atmospheric_refraction = {'refraction_mag (arcsec)':refraction_calc[:, :],\
                              'parallactic_angle (degrees)':parallactic_angle[:]}    
    return(atmospheric_refraction, input_times)
    

In [21]:
atmospheric_refraction() 


({'refraction_mag (arcsec)': array([[133.92065693, 132.11605793, 131.16761372, 130.60576893,
          130.24497771]]),
  'parallactic_angle (degrees)': array([-75.95800211])},
 <Time object: scale='utc' format='jd' value=[2459389.23219483]>)

# 4) Calculate refraction offsets in Solar NS-EW coordinate system

In [22]:
def offsets(wavelength=np.array([400, 500, 600, 700, 800])*u.nm, 
            input_times=np.array([1]), latitude=20.71*u.deg, 
            longitude=-156.25*u.deg, altitude=3055.*u.m, air_temp=20.*u.deg_C, 
            air_pressure=100000.*u.Pa, 
            humidity=75., co2_conc=380., verbose=0):

    """Computes Heliocentric shifts due to refraction

    Typical parameters for atmospheric values set as defaults.
    Computes North-South and East-West offsets in Heliocentric coordinates
    
    """
         
    refraction_atm = atmospheric_refraction(wavelength, input_times, latitude, 
                                            longitude, altitude, air_temp, 
                                            air_pressure, humidity, co2_conc, verbose)
    num_waves = wavelength.size
    
     #setting default time to now in Julian dates
    if np.alltrue(input_times == np.array([1])):
        input_times = Time(np.array([datetime.utcnow()]), scale='utc')
        input_times.format = 'jd'
    else:
        input_times = Time(input_times, format='jd', scale='utc')    
    num_times = input_times.size
    
    
    #get position angle:
    PA = coord.sun.P(refraction_atm[1]).degree
    
    parallactic_to_solar = refraction_atm[0]['parallactic_angle (degrees)'] - PA
    
    #find the offsets due to atmospheric refraction:
    sfts_heliocent_ew = np.ones((num_times, num_waves))
    sfts_heliocent_ns = np.ones((num_times, num_waves))
    
    for wv in range (num_waves):
        sfts_heliocent_ew[:, wv] = np.sin(np.radians(180 - parallactic_to_solar))
        sfts_heliocent_ns[:, wv] = np.cos(np.radians(180 - parallactic_to_solar))
    
    if verbose == 1:
        print('\nPosition Angles in degrees: ', PA, '\n') 
        for time, offsets in zip(input_times, sfts_heliocent_ew):
            print("East-West Offsets for Julian Date ", time, ": ", offsets)
        for time, offsets in zip(input_times, sfts_heliocent_ns):
            print("North-South Offsets for Julian Date ", time, ": ", offsets)
    
    offsets = {'East-West':sfts_heliocent_ew, 'North-South':sfts_heliocent_ns}
    return(offsets)

offsets(input_times = np.array([2454629.564]))
#offsets(input_times=np.array([2458278.2, 2454629.564]))

{'East-West': array([[0.99449881, 0.99449881, 0.99449881, 0.99449881, 0.99449881]]),
 'North-South': array([[0.10474783, 0.10474783, 0.10474783, 0.10474783, 0.10474783]])}

In [28]:
disp_offsets = offsets(input_times = np.linspace(0,0.5,num=11) + 2458886.4, verbose=2, air_temp=10.*u.deg_C)
#print (disp_offsets.East-West[5])
print (list(disp_offsets.keys()))
gg = disp_offsets['East-West']
print (gg[:,0])
#print ((disp_offsets['East-West']**2. ,disp_offsets['North-South']**2))
#print ((disp_offsets['East-West']**2. + disp_offsets['North-South']**2.)**0.5)
#ew_sfts = getattr(disp_offsets,'East-West')

density a -  1.2194625631296394 1.2254226533684582 0.9951362983029278
density w -  0.007078542422748544 0.009859381090734688 0.7179499765356037
n(axs):  [28274.54481324 27895.98259643 27697.04883685 27579.2438782
 27503.61875015] 
n(ws):  [318.31024401 312.25978396 309.06005724 307.135325   305.88212918] 
rho(a/axs):  0.9951362983029278 
rho(w/ws):  0.7179499765356037 
n(prop):  [28365.55669387 27984.4917631  27784.22831426 27665.61446231
 27589.45742042]
n(air):  [28137.02586165 27760.30485853 27562.33865342 27445.10666294
 27369.84935296] 
n(water):  [228.53083222 224.18690457 221.88966084 220.50779937 219.60806747]
['East-West', 'North-South']
[-0.15442054  0.31320973  0.69533048  0.88155155  0.95613682  0.99627336
  0.99668995  0.99931845  0.99993488  0.99965142  0.98590883]
